In this guided project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

In [1]:
import pandas as pd
import numpy as np
import re


autos = pd.read_csv('data/autos.csv', encoding = "Latin-1")

In [2]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [5]:
autos.rename(columns={'yearOfRegistration' : 'registration_year'}, inplace=True)
autos.rename(columns={'monthOfRegistration' : 'registration_month'}, inplace=True)
autos.rename(columns={'notRepairedDamage' : 'unrepaired_damage'}, inplace=True)
autos.rename(columns={'dateCreated' : 'ad_created'}, inplace=True)
autos.rename(columns={'kilometer' : 'odometer_km'}, inplace=True)

In [6]:
# The rest of the columnn names from camelcase to snakecase.

def convert(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    new_name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    autos.rename(columns={name : new_name}, inplace=True)

for col in autos.columns:
    convert(col)
    
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for: - Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis. - Examples of numeric data stored as text which can be cleaned and converted.

In [7]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:45:59
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


Price and odometer columns are numeric values stored as text. For each column we'll remove any non-numeric characters and convert the column to a numeric dtype.

In [8]:
autos["price"] = autos["price"].astype(int)
autos["odometer_km"] = autos["odometer_km"].astype(int)

In [9]:
# Lets see how many unique values
autos["price"].unique().shape[0]

5597

In [10]:
autos["price"].describe()

count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64

In [11]:
autos["price"].value_counts().sort_index().head(10)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
Name: price, dtype: int64

In [12]:
autos["odometer_km"].unique().shape[0]

13

In [13]:
autos["odometer_km"].describe()

count    371528.000000
mean     125618.688228
std       40112.337051
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [14]:
autos["odometer_km"].value_counts()

150000    240797
125000     38067
100000     15920
90000      12523
80000      11053
70000       9773
60000       8669
50000       7615
5000        7069
40000       6376
30000       6041
20000       5676
10000       1949
Name: odometer_km, dtype: int64

Right now, the date_crawled, last_seen, and ad_created columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.

In [15]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


In [16]:
#autos['last_seen'].value_counts(normalize=True, dropna=False).sort_index()
#autos['ad_created'].value_counts(normalize=True, dropna=False).sort_index()
#autos['date_crawled'].value_counts(normalize=True, dropna=False).sort_index()

In [17]:
autos['registration_year'].describe()

count    371528.000000
mean       2004.577997
std          92.866598
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

The registration_year column contains some odd values:

- The minimum value is 1000, before cars were invented
- The maximum value is 9999, many years into the future

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [18]:
autos["registration_year"] = autos[autos["registration_year"].between(1900,2016)]

In [19]:
autos["registration_year"].value_counts(normalize=True)

2016-03-24 14:49:47    0.000020
2016-04-02 14:50:21    0.000014
2016-03-05 14:44:30    0.000014
2016-03-07 17:36:19    0.000014
2016-04-02 21:54:36    0.000014
2016-03-08 15:50:29    0.000014
2016-03-05 14:25:23    0.000014
2016-03-23 13:50:19    0.000014
2016-03-16 13:47:44    0.000014
2016-04-02 22:54:55    0.000014
2016-03-22 14:50:05    0.000014
2016-04-01 15:56:46    0.000014
2016-03-31 18:50:16    0.000014
2016-03-29 22:50:49    0.000014
2016-03-28 10:48:11    0.000014
2016-04-04 22:38:11    0.000014
2016-04-01 19:25:23    0.000014
2016-03-14 16:51:53    0.000014
2016-03-22 15:50:32    0.000014
2016-03-09 16:48:39    0.000014
2016-03-20 16:50:22    0.000014
2016-03-11 15:36:59    0.000014
2016-03-27 12:50:22    0.000014
2016-04-03 16:49:06    0.000014
2016-03-26 22:57:31    0.000014
2016-03-09 13:50:56    0.000014
2016-03-31 16:50:28    0.000014
2016-03-19 21:49:56    0.000014
2016-03-26 10:51:07    0.000014
2016-03-21 14:50:20    0.000014
                         ...   
2016-03-

In [20]:
autos['registration_year'].describe()

count                  356780
unique                 272527
top       2016-03-24 14:49:47
freq                        7
Name: registration_year, dtype: object

When working with data on cars, it's natural to explore variations across different car brands. We can use aggregation to understand the brand column.

In [26]:
autos['brand'].value_counts().index
brands = autos['brand'].value_counts().index

For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price. 

In [29]:
mean_price_brand = {}
mean_km_brand = {}

for brand in brands:
    mean = autos.loc[autos['brand'] == brand, 'price'].mean()
    mean_price_brand[brand] = mean
    
for brand in brands:
    mean = autos.loc[autos['brand'] == brand, 'odometer_km'].mean()
    mean_km_brand[brand] = mean    


In [31]:
#mean_price_brand
#mean_km_brand

In [32]:
# Convert both dictionaries to series objects, using the series constructor
mpb_series = pd.Series(mean_price_brand)
mkb_series = pd.Series(mean_km_brand)


# Create a dataframe from the first series object
df = pd.DataFrame(mpb_series, columns=['mean_price'])

# Assign the other series as a new column in this dataframe.
df["mean_km"] = mkb_series

In [33]:
df

,mean_price,mean_km
volkswagen,14533.496007,128575.213461
bmw,14844.144883,132763.569549
opel,3223.522449,128906.592585
mercedes_benz,17244.060438,130664.703050
audi,15868.514221,129717.397256
ford,8462.054706,123839.987487
renault,2334.258946,128049.140186
peugeot,3166.981591,124970.073456
fiat,5326.312836,116854.588673
seat,4356.666050,121473.939049
